In [1]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

import ssl
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [2]:
%%skip_for_export
!jupyter nbextension enable --py tensorflow_model_analysis --system

Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK
This cell will be skipped during export to pipeline.


In [3]:
#setup paths
# This is the root directory for your TFX pip package installation.
#_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
#_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
#_serving_model_dir = os.path.join(
#    tempfile.mkdtemp(), 'serving_model/taxi_simple')
cwd = os.getcwd()
if not os.path.exists('serving_model/iris_simple'):
        os.makedirs('serving_model/iris_simple')
        
_serving_model_dir =os.path.join(cwd,'serving_model/iris_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
#_data_root = tempfile.mkdtemp(prefix='tfx-data')

if not os.path.exists(cwd+'/data/iris-data'):
        os.makedirs(cwd+'/data/iris-data') 

#if not os.path.exists(cwd+'dag_out'):
#        os.makedirs(cwd+'/dag_out')
#_dag_output = os.path.join(cwd,'dag_out')
_data_root = os.path.join(cwd,'data/iris-data')

In [5]:
#_data_root = tempfile.mkdtemp(prefix='tfx-data')
#DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
#urllib.request.urlretrieve(DATA_PATH, _data_filepath)
_data_filepath = os.path.join(_data_root, "iris.csv")

In [6]:
%%skip_for_export

!head {_data_filepath}

sepal_length,sepal_width,petal_length,petal_width,variety
5.1,3.4,1.5,0.2,0
7.7,3.0,6.1,2.3,2
5.7,2.8,4.5,1.3,1
6.8,3.2,5.9,2.3,2
5.2,3.4,1.4,0.2,0
5.6,2.9,3.6,1.3,1
5.5,2.6,4.4,1.2,1
5.5,2.4,3.7,1.0,1
4.6,3.4,1.4,0.3,0
This cell will be skipped during export to pipeline.


Create the InteractiveContext

In [7]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext(pipeline_root="./tfx_com")
#context = InteractiveContext()

Run TFX components interactively

In [8]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span = None
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.


INFO:absl:Processing input csv data /Users/amilajayatilaka/Documents/Demo/articles/AlteryxTFX/data/iris-data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./tfx_com/CsvExampleGen/examples/1, id: 2)]
        ))

In [9]:
%%skip_for_export
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] ./tfx_com/CsvExampleGen/examples/1
This cell will be skipped during export to pipeline.


In [10]:
%%skip_for_export

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "petal_length"
    value {
      float_list {
        value: 1.5
      }
    }
  }
  feature {
    key: "petal_width"
    value {
      float_list {
        value: 0.20000000298023224
      }
    }
  }
  feature {
    key: "sepal_length"
    value {
      float_list {
        value: 5.099999904632568
      }
    }
  }
  feature {
    key: "sepal_width"
    value {
      float_list {
        value: 3.4000000953674316
      }
    }
  }
  feature {
    key: "variety"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "petal_length"
    value {
      float_list {
        value: 6.099999904632568
      }
    }
  }
  feature {
    key: "petal_width"
    value {
      float_list {
        value: 2.299999952316284
      }
    }
  }
  feature {
    key: "sepal_length"
    value {
      float_list {
        value: 7.699999809265137
      }
    }
  }
  feature {
    key: "sepal_width"
    value {
      float_list {


In [11]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to ./tfx_com/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to ./tfx_com/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ./tfx_com/StatisticsGen/statistics/2, id: 3)]
        ))

In [12]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to ./tfx_com/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ./tfx_com/SchemaGen/schema/3, id: 4)]
        ))

In [14]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'petal_length',FLOAT,required,single,-
'petal_width',FLOAT,required,single,-
'sepal_length',FLOAT,required,single,-
'sepal_width',FLOAT,required,single,-
'variety',INT,required,single,-


This cell will be skipped during export to pipeline.


In [15]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to ./tfx_com/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: ./tfx_com/ExampleValidator/anomalies/4, id: 5)]
        ))

In [16]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


# Transform

In [17]:
_iris_transform_module_file = 'iris_transform.py'

In [18]:
%%skip_for_export
%%writefile {_iris_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft



def transformed_name(key):
  return key + '_xf'

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

DENSE_FLOAT_FEATURE_KEYS = ['sepal_length','sepal_width','petal_length','petal_width'
                           ]


# Keys
LABEL_KEY = 'variety'

_DENSE_FLOAT_FEATURE_KEYS = DENSE_FLOAT_FEATURE_KEYS
_transformed_name = transformed_name
_LABEL_KEY = LABEL_KEY

def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}

  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
  # TODO(b/157064428): Support label transformation for Keras.
  # Do not apply label transformation as it will result in wrong evaluation.
  outputs[_transformed_name(_LABEL_KEY)] = _fill_in_missing(inputs[_LABEL_KEY])

  return outputs


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting iris_transform.py
This cell will be skipped during export to pipeline.


In [19]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_iris_transform_module_file))
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


INFO:absl:Using 1 process(es) for Beam pipeline execution.


  name: "petal_length"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "petal_width"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "sepal_length"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "sepal_width"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "variety"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
}) belongs to. Thus noop.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ./tfx_com/Transform/transform_graph/5/.temp_path/tftransform_tmp/60b8caa1f855436da40879044e0772c3/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ./tfx_com/Transform/transform_graph/5/.temp_path/tftransform_tmp/e4593fb7f51d4bd1be1f1ecbc247c39d

...

  name: "petal_length"
  type: FLOAT
}
feature {
  name: "petal_width"
  type: FLOAT
}
feature {
  name: "sepal_length"
  type: FLOAT
}
feature {
  name: "sepal_width"
  type: FLOAT
}
feature {
  name: "variety"
  type: INT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7ff08e317c88>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "petal_length_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "petal_width_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "sepal_length_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "sepal_width_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "variety_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0x7ff08e32b908>))) belongs to. T

  name: "petal_length"
  type: FLOAT
}
feature {
  name: "petal_width"
  type: FLOAT
}
feature {
  name: "sepal_length"
  type: FLOAT
}
feature {
  name: "sepal_width"
  type: FLOAT
}
feature {
  name: "variety"
  type: INT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7ff08e317c88>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "petal_length_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "petal_width_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "sepal_length_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "sepal_width_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "variety_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0x7ff08e32b908>))) belongs to. T

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tfx_com/Transform/transform_graph/5/.temp_path/tftransform_tmp/f1ee2d106a104e9a8798a09344524ea2/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ./tfx_com/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./tfx_com/Transform/transformed_examples/5, id: 7)]
        ))

In [20]:
%%skip_for_export

transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: ./tfx_com/Transform/transform_graph/5, id: 6)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: ./tfx_com/Transform/transformed_examples/5, id: 7)]
)}

This cell will be skipped during export to pipeline.


In [21]:
%%skip_for_export

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "petal_length_xf"
    value {
      float_list {
        value: -1.3022433519363403
      }
    }
  }
  feature {
    key: "petal_width_xf"
    value {
      float_list {
        value: -1.3272178173065186
      }
    }
  }
  feature {
    key: "sepal_length_xf"
    value {
      float_list {
        value: -0.8411077857017517
      }
    }
  }
  feature {
    key: "sepal_width_xf"
    value {
      float_list {
        value: 0.9621586203575134
      }
    }
  }
  feature {
    key: "variety_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "petal_length_xf"
    value {
      float_list {
        value: 1.3528157472610474
      }
    }
  }
  feature {
    key: "petal_width_xf"
    value {
      float_list {
        value: 1.4519989490509033
      }
    }
  }
  feature {
    key: "sepal_length_xf"
    value {
      float_list {
        value: 2.2660958766937256
      }
    }
  }
  feature {
    key: 

In [22]:
_iris_trainer_module_file = 'iris_trainer_k.py'

In [23]:
%%skip_for_export
%%writefile {_iris_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft
import kerastuner
from tensorflow import keras

from tfx.components.trainer.executor import TrainerFnArgs



def transformed_name(key):
  return key + '_xf'

def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]

def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

DENSE_FLOAT_FEATURE_KEYS = ['sepal_length','sepal_width','petal_length','petal_width'
                           ]


# Keys
LABEL_KEY = 'variety'

_DENSE_FLOAT_FEATURE_KEYS = DENSE_FLOAT_FEATURE_KEYS
_transformed_name = transformed_name
_LABEL_KEY = LABEL_KEY

# Iris dataset has 150 records, and is divided to train and eval splits in 2:1
# ratio.
_TRAIN_DATA_SIZE = 100
_EVAL_DATA_SIZE = 50
_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn

def _input_fn(file_pattern: List[Text],
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key=_transformed_name(_LABEL_KEY))

  return dataset

def _get_hyperparameters() -> kerastuner.HyperParameters:
  """Returns hyperparameters for building Keras model."""
  hp = kerastuner.HyperParameters()
  # Defines search space.
  hp.Choice('learning_rate', [1e-2, 1e-3], default=1e-2)
  hp.Int('num_layers', 1, 3, default=2)
  return hp

def _build_keras_model(hparams: kerastuner.HyperParameters) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying iris data.
  Args:
    hparams: Holds HyperParameters for tuning.
  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=_transformed_name(f))
      for f in _DENSE_FLOAT_FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(int(hparams.get('num_layers'))):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3, activation='softmax')(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(hparams.get('learning_rate')),
      loss='sparse_categorical_crossentropy',
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=absl.logging.info)
  return model

# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.
  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, tf_transform_output,
                            batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output,
                           batch_size=_EVAL_BATCH_SIZE)

  if fn_args.hyperparameters:
    hparams = kerastuner.HyperParameters.from_config(fn_args.hyperparameters)
  else:
    # This is a shown case when hyperparameters is decided and Tuner is removed
    # from the pipeline. User can also inline the hyperparameters directly in
    # _build_keras_model.
    hparams = _get_hyperparameters()
  absl.logging.info('HyperParameters for training: %s' % hparams.get_config())

  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():
    model = _build_keras_model(hparams)

  steps_per_epoch = _TRAIN_DATA_SIZE / _TRAIN_BATCH_SIZE

  try:
    log_dir = fn_args.model_run_dir
  except KeyError:
    # TODO(b/158106209): use ModelRun instead of Model artifact for logging.
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

  # Write logs to path
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, update_freq='batch')

  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / steps_per_epoch),
      steps_per_epoch=steps_per_epoch,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])


  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting iris_trainer_k.py
This cell will be skipped during export to pipeline.


In [24]:
trainer = Trainer(
    module_file=os.path.abspath(_iris_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=250))
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:HyperParameters for training: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'values': [0.01, 0.001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'num_layers', 'default': 2, 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}}], 'values': {'learning_rate': 0.01, 'num_layers': 2}}


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:sepal_length_xf (InputLayer)    [(None, 1)]          0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:sepal_width_xf (InputLayer)     [(None, 1)]          0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:petal_length_xf (InputLayer)    [(None, 1)]          0                                            
INFO:absl:_____________________________________________________________________________________________

Epoch 1/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.0500 - loss: 1.190 - 2s 349ms/step - sparse_categorical_accuracy: 0.2400 - loss: 1.1107 - val_sparse_categorical_accuracy: 0.7104 - val_loss: 0.9785
Epoch 2/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.7000 - loss: 0.992 - 1s 101ms/step - sparse_categorical_accuracy: 0.7600 - loss: 0.9468 - val_sparse_categorical_accuracy: 0.8672 - val_loss: 0.8052
Epoch 3/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.8500 - loss: 0.862 - 1s 125ms/step - sparse_categorical_accuracy: 0.8400 - loss: 0.7937 - val_sparse_categorical_accuracy: 0.8448 - val_loss: 0.6373
Epoch 4/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.8000 - loss: 0.676 - 1s 134ms/step - sparse_categorical_accuracy: 0.8300 - loss: 0.6211 - val_sparse_categorical_accuracy: 0.8444 - val_loss: 0.4891
Epoch 5/100
5/5 [===================

Epoch 35/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.045 - 0s 76ms/step - sparse_categorical_accuracy: 0.9700 - loss: 0.0555 - val_sparse_categorical_accuracy: 0.9332 - val_loss: 0.0967
Epoch 36/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.037 - 0s 75ms/step - sparse_categorical_accuracy: 0.9900 - loss: 0.0402 - val_sparse_categorical_accuracy: 0.9336 - val_loss: 0.0986
Epoch 37/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.021 - 0s 74ms/step - sparse_categorical_accuracy: 0.9900 - loss: 0.0327 - val_sparse_categorical_accuracy: 0.9336 - val_loss: 0.0887
Epoch 38/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.060 - 0s 75ms/step - sparse_categorical_accuracy: 0.9700 - loss: 0.0754 - val_sparse_categorical_accuracy: 0.9560 - val_loss: 0.0834
Epoch 39/100
5/5 [==================

Epoch 69/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 0.9500 - loss: 0.050 - 0s 79ms/step - sparse_categorical_accuracy: 0.9800 - loss: 0.0436 - val_sparse_categorical_accuracy: 0.9336 - val_loss: 0.1294
Epoch 70/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.032 - 0s 90ms/step - sparse_categorical_accuracy: 0.9900 - loss: 0.0413 - val_sparse_categorical_accuracy: 0.9556 - val_loss: 0.0938
Epoch 71/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.047 - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.021 - 0s 91ms/step - sparse_categorical_accuracy: 0.9900 - loss: 0.0408 - val_sparse_categorical_accuracy: 0.9336 - val_loss: 0.1121
Epoch 72/100
5/5 [==============================] - ETA: 0s - sparse_categorical_accuracy: 1.0000 - loss: 0.024 - 0s 81ms/step - sparse_categorical_accuracy: 0.9900 - loss: 0.0351 - val_sparse_categorical_accuracy: 

INFO:tensorflow:Assets written to: ./tfx_com/Trainer/model/6/serving_model_dir/assets


INFO:absl:Training complete. Model written to ./tfx_com/Trainer/model/6/serving_model_dir
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ./tfx_com/Trainer/model/6, id: 8)]
        ))

In [25]:
%%skip_for_export
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

['serving_model_dir', 'logs']
['variables', 'saved_model.pb', 'assets']
This cell will be skipped during export to pipeline.


In [26]:
%%skip_for_export

log_dir = os.path.join(model_artifact_dir, 'logs')

%load_ext tensorboard
%tensorboard --logdir {log_dir}
%reload_ext tensorboard

This cell will be skipped during export to pipeline.


In [27]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [28]:
import tensorflow_transform as tft
eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='variety')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
          ])
      ])

In [29]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    #baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "variety"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
}

INFO:absl:Using ./tfx_com/Trainer/model/6/serving_model_dir as  model.
INFO:absl:Evaluating model.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Evaluation complete. Results written to ./tfx_com/Evaluator/evaluation/7.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to ./tfx_com/Evaluator/blessing/7.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: ./tfx_com/Evaluator/evaluation/7, id: 9)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: ./tfx_com/Evaluator/blessing/7, id: 10)]
        ))

In [30]:
%%skip_for_export

evaluator.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(type_name: ModelEvaluation, uri: ./tfx_com/Evaluator/evaluation/7, id: 9)]
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(type_name: ModelBlessing, uri: ./tfx_com/Evaluator/blessing/7, id: 10)]
)}

This cell will be skipped during export to pipeline.


In [31]:
%%skip_for_export

context.show(evaluator.outputs['evaluation'])

This cell will be skipped during export to pipeline.


In [32]:
%%skip_for_export

blessing_uri = evaluator.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r--  1 amilajayatilaka  staff  0 28 Jul 06:57 BLESSED
This cell will be skipped during export to pipeline.


In [33]:
%%skip_for_export

PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

validation_ok: true

This cell will be skipped during export to pipeline.


In [34]:
%%skip_for_export

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column petal_width.
#tfma_eval = tfma.view.render_slicing_metrics(tfma_result, slicing_column='petal_width')
#tfma.view.render_plot(tfma_result)
#print(type(tfma_eval))


This cell will be skipped during export to pipeline.


In [35]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1595883420
INFO:absl:Model written to serving path /Users/amilajayatilaka/Documents/Demo/articles/AlteryxTFX/serving_model/iris_simple/1595883420.
INFO:absl:Model pushed to ./tfx_com/Pusher/pushed_model/8.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 8
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: ./tfx_com/Pusher/pushed_model/8, id: 11)]
        ))

In [36]:
%%skip_for_export

pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(type_name: PushedModel, uri: ./tfx_com/Pusher/pushed_model/8, id: 11)]
)}

This cell will be skipped during export to pipeline.


In [37]:
%%skip_for_export
print(pusher.outputs.model_push.get()[0].uri)

./tfx_com/Pusher/pushed_model/8
This cell will be skipped during export to pipeline.


In [38]:
#!ls tfx_com/Pusher/pushed_model/8

assets         saved_model.pb variables


In [39]:
%%skip_for_export

push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <tensorflow.python.saved_model.load._WrapperFunction object at 0x7ff08d108438>)
This cell will be skipped during export to pipeline.


In [40]:
_runner_type = 'airflow' #@param ["beam", "airflow"]
_pipeline_name = 'iris_%s' % _runner_type

In [41]:
#docs_infra: no_execute
# For Colab notebooks only.
# TODO(USER): Fill out the path to this notebook.
#_notebook_filepath = (
#    '/content/drive/My Drive/Colab Notebooks/components.ipynb')

# For Jupyter notebooks only.
_notebook_filepath = os.path.join(os.getcwd(),
                                   'iris_example.ipynb')

# TODO(USER): Fill out the paths for the exported pipeline.
#_tfx_root = os.path.join(_dag_output, 'tfx')
_iris_root = os.path.join(cwd, 'iris')
_serving_model_dir = os.path.join(_serving_model_dir, 'serving_model')
_data_root = os.path.join(cwd, 'data', 'simple')
_pipeline_root = os.path.join(cwd, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(cwd, 'metadata', _pipeline_name,
                              'metadata.db')

In [42]:
#docs_infra: no_execute
# TODO(USER): Specify components to be included in the exported pipeline.
components = [
    example_gen, statistics_gen, schema_gen, example_validator, transform,
    trainer, evaluator, pusher
]

In [43]:
%%skip_for_export
#docs_infra: no_execute

#@markdown Run this cell to generate the pipeline files.

if get_ipython().magics_manager.auto_magic:
  print('Warning: %automagic is ON. Line magics specified without the % prefix '
        'will not be scrubbed during export to pipeline.')

_pipeline_export_filepath = 'export_%s.py' % _pipeline_name
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=_pipeline_export_filepath,
                           runner_type=_runner_type)

INFO:absl:Exporting contents of /Users/amilajayatilaka/Documents/Demo/articles/AlteryxTFX/iris_example.ipynb to export_iris_airflow.py with airflow runner.
INFO:absl:25 cell(s) marked with "%%skip_for_export", skipped.


This cell will be skipped during export to pipeline.


In [44]:
%%skip_for_export
#!jupyter nbextension enable --py widgetsnbextension --system
#!jupyter nbextension install --py --symlink tensorflow_model_analysis --system


This cell will be skipped during export to pipeline.


In [45]:
%%skip_for_export
#!pip freeze -> requirements.txt

This cell will be skipped during export to pipeline.


In [46]:
%%skip_for_export
#!pip3 install tensorflow==2.2.0
#!pip3 install tensorboard>=2.1,<2.3
#!pip3 install tensorflow-data-validation==0.22.0
#!pip3 install tensorflow-metadata==0.22.0
#!pip3 install tensorflow-model-analysis==0.22.1
#!pip3 install tensorflow-transform==0.22.0
#!pip3 install tfx-bsl==0.22.0

This cell will be skipped during export to pipeline.


In [47]:
#!pip3 install ipywidgets==7.5.0

In [48]:
#!pip3 install -U tensorboard-plugin-profile

In [49]:
#!pip3 install tfx==0.22.0